In [1]:
import boto3
import json, uuid, copy, datetime
import random, names, tabulate

In [2]:
# open_cmd_in_new_terminal("docker-compose up")
!gnome-terminal -- docker-compose up

In [3]:
!./setuprds.sh
!./setupdynamo.sh

Setting up users
CREATE ROLE
CREATE ROLE
CREATE ROLE
Setting up account ledger in RDS local
CREATE SCHEMA
CREATE TABLE
CREATE INDEX
CREATE INDEX
REVOKE
GRANT
GRANT
GRANT
GRANT
GRANT
GRANT
Setting up transaction ledger in RDS local
CREATE SCHEMA
CREATE TABLE
REVOKE
GRANT
GRANT
GRANT
GRANT
GRANT
Setting up float ledger in RDS local
CREATE SCHEMA
CREATE TABLE
REVOKE
GRANT
GRANT
GRANT
GRANT
GRANT
Creating the DynamoDB table that holds the client floats
{
    "StackId": "arn:aws:cloudformation:us-east-1:123456789:stack/client-float-table/9c21165f-9fd2-4704-ad3e-a3539acbbbba"
}
Adding in item for core, ZAR wholesale float


In [4]:
!./deploylambdas.sh

Deploying the float lambda
Serverless: config.options_stage: local
Serverless: serverless.service.custom.stage: undefined
Serverless: serverless.service.provider.stage: dev
Serverless: config.stage: local
Serverless: Using serverless-localstack
Serverless: Reconfiguring service apigateway to use http://localhost:4567
Serverless: Reconfiguring service cloudformation to use http://localhost:4581
Serverless: Reconfiguring service cloudwatch to use http://localhost:4582
Serverless: Reconfiguring service lambda to use http://localhost:4574
Serverless: Reconfiguring service dynamodb to use http://localhost:4569
Serverless: Reconfiguring service kinesis to use http://localhost:4568
Serverless: Reconfiguring service route53 to use http://localhost:4580
Serverless: Reconfiguring service firehose to use http://localhost:4573
Serverless: Reconfiguring service stepfunctions to use http://localhost:4585
Serverless: Reconfiguring service es to use http://localhost:4578
Serverless: Reconfiguring serv

In [5]:
dynamodb = boto3.client('dynamodb', endpoint_url = 'http://localhost:4569')
local_lambda = boto3.client('lambda', endpoint_url = 'http://localhost:4574')

In [6]:
def print_deployed_functions():
    function_list = local_lambda.list_functions()
    # print(function_list)
    print('Function list: ', [function['FunctionName'] for function in function_list['Functions']])

In [7]:
def generate_account(client_id = 'zar_client_co', float_id = 'zar_cash_float'):
    first_name = names.get_first_name()
    family_name = names.get_last_name()
    user_id = str(uuid.uuid4())
    return { 'clientId': client_id, 'floatId': float_id, 'ownerUserId': user_id, 'userFirstName': first_name, 'userFamilyName': family_name}

In [8]:
def decode_lambda_result(lambda_result):
    lambda_payload = lambda_result['Payload'].read()
    lambda_pload_decoded = lambda_payload.decode('utf-8')
    lambda_pload_object = json.loads(lambda_pload_decoded)
    return lambda_pload_object

In [9]:
def create_number_accounts(number_accounts = 1, client_id = 'zar_client_co'):
    account_dicts = [generate_account(client_id) for i in range(number_accounts)]
    persisted_accounts = []
    for account in account_dicts:
        lambda_result = local_lambda.invoke(FunctionName='create-account', InvocationType='RequestResponse', 
                                           Payload=json.dumps(account))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_account = copy.deepcopy(account)
        persisted_account['accountId'] = lambda_payload['accountId']
        persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Account with ID %s persisted at %s' % (persisted_account['accountId'], persisted_account['persistedTime']))
        persisted_accounts.append(persisted_account)
    
    return persisted_accounts
    

In [22]:
def generate_saving_transaction(account_id, ref_amount = 100, float_id = 'zar_cash_primary'):
    current_time = datetime.datetime.now()
    saved_amount = round(random.random() * ref_amount * 10 * 10) # random proportion of ref amount, 
    test_saving_dict = { 
        'accountId': account_id, 
        'initiationTime': str(current_time), 
        'settlementTime': str(current_time), 
        'savedAmount': saved_amount, 
        'savedCurrency': 'ZAR', 
        'savedUnit': 'HUNDREDTH_CENT',
        'floatId': float_id
    }
    return test_saving_dict

In [42]:
def seed_savings_for_accounts(accounts, tx_per_account = 1, base_amount = 100):
    transactions = []
    for i in range(tx_per_account):
        transactions.extend([generate_saving_transaction(account['accountId'], base_amount) for account in accounts])
#     print('transactions: ', transactions)
    tx_records = []
    for tx in transactions:
        lambda_result = local_lambda.invoke(FunctionName='add-savings', InvocationType='RequestResponse', 
                                           Payload=json.dumps(tx, default=str))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_tx = copy.deepcopy(tx)
        persisted_tx['transactionId'] = lambda_payload['transactionDetails'][0][0]['transaction_id']
        persisted_tx['currentBalance'] = lambda_payload['newBalance']
        # persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Transaction persisted with ID %s' % (persisted_tx['transactionId']))
        tx_records.append(persisted_tx)
    
    return tx_records

In [12]:
print_deployed_functions()

Function list:  ['float-api-local-accrue', 'create-account', 'add-savings']


In [13]:
accounts = create_number_accounts(number_accounts = 20)

Account with ID 987d21b0-6353-49c5-91e6-fc842220a1a9 persisted at 2019-05-29T12:45:51.693Z
Account with ID d7167b20-1761-4ce6-a824-72cc0e547105 persisted at 2019-05-29T12:45:52.697Z
Account with ID 4fcb7361-8f8a-4e54-b4f2-39fcddd21ea2 persisted at 2019-05-29T12:45:53.195Z
Account with ID 30271a08-21f9-47ad-a2e1-fc4c631de02c persisted at 2019-05-29T12:45:53.753Z
Account with ID 5ec9cf7f-1760-4ad9-94ae-271c82e6ba9f persisted at 2019-05-29T12:45:54.290Z
Account with ID d2a1d3d5-7427-40b7-b09a-8f6734c5a78e persisted at 2019-05-29T12:45:54.813Z
Account with ID 5a845113-51c2-42ba-9548-4be9994d08ab persisted at 2019-05-29T12:45:55.342Z
Account with ID 93196722-686c-47e3-94c6-b08e676b755d persisted at 2019-05-29T12:45:55.864Z
Account with ID 6408d593-2b09-4653-bf28-ba6db2bc2919 persisted at 2019-05-29T12:45:56.361Z
Account with ID 5f07f4bf-8a62-4141-adc9-e291beaa58d7 persisted at 2019-05-29T12:45:56.872Z
Account with ID 84c52f59-d483-42f4-ab73-2f1919d84a83 persisted at 2019-05-29T12:45:57.383Z

In [14]:
account_info = [{ 'Name': account['userFirstName'] + ' ' + account['userFamilyName'], 
                'AccountId': account['accountId'] } for account in accounts]
print(tabulate.tabulate(account_info, headers = 'keys'))

Name              AccountId
----------------  ------------------------------------
Karl Warner       987d21b0-6353-49c5-91e6-fc842220a1a9
Joshua Stout      d7167b20-1761-4ce6-a824-72cc0e547105
Jose Holder       4fcb7361-8f8a-4e54-b4f2-39fcddd21ea2
Fannie Warren     30271a08-21f9-47ad-a2e1-fc4c631de02c
Garry Oneill      5ec9cf7f-1760-4ad9-94ae-271c82e6ba9f
Paula Kimes       d2a1d3d5-7427-40b7-b09a-8f6734c5a78e
Raymond Caminiti  5a845113-51c2-42ba-9548-4be9994d08ab
Edgar Coca        93196722-686c-47e3-94c6-b08e676b755d
Francisco Boren   6408d593-2b09-4653-bf28-ba6db2bc2919
Kelly Rodman      5f07f4bf-8a62-4141-adc9-e291beaa58d7
Theresa Garcia    84c52f59-d483-42f4-ab73-2f1919d84a83
Gary Jolly        bc2053da-3a9c-408e-9e2d-f12f2ae70b70
Mark Mcdonald     c74f0608-4df4-45e1-9806-668b2cdb324c
Barbara Reed      400b06a9-bcdb-455e-8c61-db8aa0dafc12
Jennifer Dejesus  36d0fdc1-974e-4a80-bbed-e426ebf3ac04
Samantha Trumbo   0b543593-533a-496a-8e36-0aa10ad7e6de
Barbara Mcdaniel  253f7ee8-37b5-424a-

In [36]:
single_tx = generate_saving_transaction(accounts[0]['accountId'], 100)
lambda_result = local_lambda.invoke(FunctionName='add-savings', Payload=json.dumps(single_tx, default=str))
result_decoded = decode_lambda_result(lambda_result)
print('Lambda result', result_decoded)

Lambda result {'body': '{"transactionDetails":[[{"transaction_id":"22116c27-8647-45e0-ad07-94d0052e74c3","tags":[],"flags":[]}],[{"transaction_id":"7549d020-c254-4967-9e27-de534ff34577","creation_time":"2019-05-29T13:40:39.641Z"}]],"newBalance":"15856"}', 'statusCode': 200}


In [48]:
account_balances = seed_savings_for_accounts(accounts, 1, 100)

transactions:  [{'accountId': '987d21b0-6353-49c5-91e6-fc842220a1a9', 'initiationTime': '2019-05-29 15:59:54.985508', 'settlementTime': '2019-05-29 15:59:54.985508', 'savedAmount': 7018, 'savedCurrency': 'ZAR', 'savedUnit': 'HUNDREDTH_CENT', 'floatId': 'zar_cash_primary'}, {'accountId': 'd7167b20-1761-4ce6-a824-72cc0e547105', 'initiationTime': '2019-05-29 15:59:54.985790', 'settlementTime': '2019-05-29 15:59:54.985790', 'savedAmount': 5320, 'savedCurrency': 'ZAR', 'savedUnit': 'HUNDREDTH_CENT', 'floatId': 'zar_cash_primary'}, {'accountId': '4fcb7361-8f8a-4e54-b4f2-39fcddd21ea2', 'initiationTime': '2019-05-29 15:59:54.985797', 'settlementTime': '2019-05-29 15:59:54.985797', 'savedAmount': 216, 'savedCurrency': 'ZAR', 'savedUnit': 'HUNDREDTH_CENT', 'floatId': 'zar_cash_primary'}, {'accountId': '30271a08-21f9-47ad-a2e1-fc4c631de02c', 'initiationTime': '2019-05-29 15:59:54.985802', 'settlementTime': '2019-05-29 15:59:54.985802', 'savedAmount': 4482, 'savedCurrency': 'ZAR', 'savedUnit': 'HU

In [50]:
print([account['currentBalance'] for account in account_balances])

['62834', '5320', '216', '4482', '1715', '9894', '7012', '6173', '1091', '9280', '3604', '2469', '354', '5785', '7372', '4995', '2184', '5000', '7796', '8779']
